# AutoGenとChromaDBを使用したAgentic RAG

この例では、ChromaDBをベクトルデータベースとしてAutoGenフレームワークと統合し、Retrieval-Augmented Generation（RAG）を実装する方法を示します。

> **📝 注意**: このノートブックでは**単一エージェント**を使用したRAG実装を学習します。真の「Agentic RAG」（複数エージェントの協調）への入門段階として位置づけられます。

## 📚 このノートブックで学習できること

### 🎯 主要な学習目標
- **AutoGenフレームワーク**: マルチエージェントシステムの基礎と実装
- **ChromaDB**: ベクトルデータベースを使用したセマンティック検索
- **RAG（Retrieval-Augmented Generation）**: 外部データを活用したAI応答生成
- **Azure AI統合**: GitHub TokenによるAzure AI Model Inference APIの活用

### 🛠️ 実装する技術スタック
- **ベクトルデータベース**: ChromaDBによるドキュメント保存・検索
- **AIエージェント**: AssistantAgentによる知識統合型応答
- **コンテキスト統合**: 複数データソース（文書+天気）の組み合わせ
- **非同期処理**: Python asyncioによる効率的なデータ処理

### 💡 実用的なスキル
- 企業ドキュメントの効率的な検索システム構築
- AIエージェントを活用したカスタマーサポート自動化
- マルチソース情報統合による高度なQAシステム開発

## ⚠️ トラブルシューティング

> **注意**: SQLiteバージョンエラーについて

次のエラーが発生した場合：
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```

**対処方法**: 下記のセル4（コメントアウトされたコード）のコメントを外して実行してください：

> 💡 **補足**: この問題は古いPython環境でChromaDBを使用する際に発生することがあります。通常の環境では必要ありません。

In [43]:
import importlib
import subprocess
import sys

# チェックするライブラリ名と、インストールするパッケージ名のマッピング
packages = {
    "autogen_agentchat": "autogen-agentchat",
    "autogen_core": "autogen-core", 
    "autogen_ext": "autogen-ext[azure]",
    "chromadb": "chromadb",
    "azure.core": "azure-core",
    "dotenv": "python-dotenv",
}

not_installed = []
for lib, pkg in packages.items():
    try:
        # ライブラリがインポート可能か試す
        importlib.import_module(lib.split('.')[0])
    except ImportError:
        # インポートできなければインストールリストに追加
        not_installed.append(pkg)

if not_installed:
    print(f"不足しているパッケージをインストールします: {', '.join(not_installed)}")
    # pipコマンドを実行してインストール
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet"] + not_installed)
    print("パッケージのインストールが完了しました。")
else:
    print("必要なパッケージはすべてインストール済みです。")

必要なパッケージはすべてインストール済みです。


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [44]:
import os
import time
import asyncio
import warnings
from typing import List, Dict
from dotenv import load_dotenv

# 不要な警告を非表示にする
warnings.filterwarnings("ignore", message=".*Protobuf gencode version.*")
warnings.filterwarnings("ignore", message=".*Missing required field 'structured_output'.*")
warnings.filterwarnings("ignore", message=".*Unclosed client session.*")

from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from azure.core.credentials import AzureKeyCredential
from autogen_ext.models.azure import AzureAIChatCompletionClient

import chromadb

load_dotenv()

True

## クライアントの作成

まず、Azure AI Chat Completion Clientを初期化します。このクライアントは、Azure OpenAIサービスとやり取りしてユーザークエリへの応答を生成するために使用されます。

In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    # Azure AI Model Inference API - AutoGenとAzureモデルを統合するための統一エンドポイント
    # GitHub Tokenでの認証をサポートし、複数のAIモデルに統一されたAPIでアクセス可能
    endpoint="https://models.inference.ai.azure.com",
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "structured_output": True,  # 警告を解決するために追加
        "family": "unknown",
    },
)

## ベクトルデータベースの初期化

永続ストレージを使用してChromaDBを初期化し、拡張されたサンプルドキュメントを追加します。ChromaDBは、正確な応答生成のためのコンテキストを提供するドキュメントの保存と検索に使用されます。

In [47]:
# ChromaDBを永続ストレージで初期化
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.create_collection(
    name="travel_documents",
    metadata={"description": "travel_service"},
    get_or_create=True
)

# 拡張されたサンプルドキュメント（日本語版）
documents = [
    "Contoso旅行では、世界各地のエキゾチックな目的地への高級バケーションパッケージを提供しています。",
    "当社のプレミアム旅行サービスには、個人向け旅程計画と24時間365日のコンシェルジュサポートが含まれています。",
    "Contosoの旅行保険は、医療緊急事態、旅行キャンセル、手荷物紛失をカバーしています。",
    "人気の目的地には、モルディブ、スイスアルプス、アフリカサファリなどがあります。",
    "Contoso旅行では、ブティックホテルやプライベートガイドツアーへの限定アクセスを提供しています。"
]

# メタデータと共にドキュメントを追加
collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"source": "training", "type": "explanation"} for _ in documents]
)

## コンテキストプロバイダーの実装

`ContextProvider`クラスは複数のソースからのコンテキスト取得と統合を処理します：

1. **ベクトルデータベース検索**: ChromaDBを使用して旅行ドキュメントのセマンティック検索を実行
2. **天気情報**: 主要都市のシミュレートされた天気データベースを維持
3. **統合コンテキスト**: ドキュメントと天気データの両方を包括的なコンテキストに結合

主要メソッド:
- `get_retrieval_context()`: クエリに基づいて関連ドキュメントを取得
- `get_weather_data()`: 指定された場所の天気情報を提供
- `get_unified_context()`: ドキュメントと天気の両方のコンテキストを組み合わせて拡張された応答を実現

In [48]:
class ContextProvider:
    def __init__(self, collection):
        self.collection = collection
        # シミュレートされた天気データベース
        self.weather_database = {
            "new york": {"temperature": 22, "condition": "曇り時々晴れ", "humidity": 65, "wind": "時速16km"},
            "london": {"temperature": 16, "condition": "雨", "humidity": 80, "wind": "時速24km"},
            "tokyo": {"temperature": 24, "condition": "晴れ", "humidity": 50, "wind": "時速8km"},
            "sydney": {"temperature": 27, "condition": "快晴", "humidity": 45, "wind": "時速19km"},
            "paris": {"temperature": 20, "condition": "曇り", "humidity": 70, "wind": "時速13km"},
        }
    
    def get_retrieval_context(self, query: str) -> str:
        """クエリに基づいてベクトルデータベースから関連ドキュメントを取得します。"""
        results = self.collection.query(
            query_texts=[query],
            include=["documents", "metadatas"],
            n_results=2
        )
        context_strings = []
        if results and results.get("documents") and len(results["documents"][0]) > 0:
            for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
                context_strings.append(f"Document: {doc}\nMetadata: {meta}")
        return "\n\n".join(context_strings) if context_strings else "No relevant documents found"
    
    def get_weather_data(self, location: str) -> str:
        """指定された場所の天気データを取得をシミュレートします。"""
        if not location:
            return ""
            
        location_key = location.lower()
        if location_key in self.weather_database:
            data = self.weather_database[location_key]
            return f"{location.title()}の天気:\n" \
                   f"気温: {data['temperature']}°C\n" \
                   f"天候: {data['condition']}\n" \
                   f"湿度: {data['humidity']}%\n" \
                   f"風速: {data['wind']}"
        else:
            return f"{location}の天気データは利用できません。"
    
    def get_unified_context(self, query: str, location: str = None) -> str:
        """ドキュメント検索と天気データの両方を組み合わせた統合コンテキストを返します。"""
        retrieval_context = self.get_retrieval_context(query)
        
        weather_context = ""
        if location:
            weather_context = self.get_weather_data(location)
            weather_intro = f"\n{location}の天気情報:\n"
        else:
            weather_intro = ""
        
        return f"取得されたコンテキスト:\n{retrieval_context}\n\n{weather_intro}{weather_context}"

## エージェント設定

検索エージェントとアシスタントエージェントを設定します。検索エージェントはセマンティック検索を使用して関連情報を見つけることに特化し、アシスタントは取得された情報に基づいて詳細な応答を生成します。

In [49]:
# 拡張機能を持つエージェントを作成
assistant = AssistantAgent(
    name="assistant",
    model_client=client,
    system_message=(
        "あなたは提供されたコンテキストの情報を活用して回答する親切なAIアシスタントです。"
        "コンテキストには文書検索結果と天気情報の両方が含まれる場合があります。"
        "提供されたすべての情報を確認し、利用可能な情報に基づいて適切に回答してください。"
        "天気情報が提供されている場合は、その情報を使用して天気に関する質問に答えてください。"
        "情報が提供されていない場合のみ、「情報がありません」と回答してください。"
    ),
)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000002A4076D96D0>


## RAGを使用したクエリ処理

`ask_rag`関数を定義して、アシスタントにクエリを送信し、応答を処理し、評価します。この関数はアシスタントとのやり取りを処理し、評価器を使用して応答の品質を測定します。

In [50]:
async def ask_rag_agent(query: str, context_provider: ContextProvider, location: str = None):
    """
    コンテキストプロバイダーからのコンテキストと共にアシスタントエージェントにクエリを送信します。
    
    Args:
        query: ユーザーの質問
        context_provider: コンテキストプロバイダーのインスタンス
        location: 天気クエリ用のオプションの場所
    """
    try:
        # 統合コンテキストを取得
        context = context_provider.get_unified_context(query, location)
        
        # クエリをコンテキストで拡張
        augmented_query = (
            f"以下の情報が利用可能です:\n{context}\n\n"
            f"ユーザークエリ: {query}\n\n"
            "上記の利用可能な情報をすべて確認し、文書情報と天気情報の両方を適切に活用して回答してください。"
        )

        # 拡張されたクエリをアシスタントに送信
        start_time = time.time()
        response = await assistant.on_messages(
            [TextMessage(content=augmented_query, source="user")],
            cancellation_token=CancellationToken(),
        )
        processing_time = time.time() - start_time
        
        return {
            'query': query,
            'response': response.chat_message.content,
            'processing_time': processing_time,
            'location': location
        }
    except Exception as e:
        print(f"クエリ処理中にエラーが発生しました: {e}")
        return None

# 使用例

コンテキストプロバイダーを初期化し、処理したいクエリを定義します。

In [51]:
async def main():
    # コンテキストプロバイダーを初期化
    context_provider = ContextProvider(collection)
    
    # サンプルクエリ
    queries = [
        {"query": "Contosoの旅行保険は何をカバーしていますか？"},
        {"query": "シドニーの天気はどうですか？", "location": "sydney"},
        {"query": "Contosoが提供する高級な目的地は何ですか？パリの天気も教えてください。", "location": "paris"},
    ]
    
    print("=== AutoGen RAGデモ ===")
    for query_data in queries:
        query = query_data["query"]
        location = query_data.get("location")
        
        print(f"\n\nクエリ: {query}")
        if location:
            print(f"場所: {location}")
        
        # 使用されているコンテキストを表示
        context = context_provider.get_unified_context(query, location)
        print("\n--- 使用されたコンテキスト ---")
        print(context)
        print("------------------------")
        
        # エージェントから応答を取得
        result = await ask_rag_agent(query, context_provider, location)
        if result:
            print(f"\n応答: {result['response']}")
        print("\n" + "="*50)

## スクリプトの実行

スクリプトがインタラクティブ環境または標準スクリプトで実行されているかを確認し、それに応じてmain関数を実行します。

In [52]:
if __name__ == "__main__":
    if asyncio.get_event_loop().is_running():
        await main()
    else:
        asyncio.run(main())

=== AutoGen RAGデモ ===


クエリ: Contosoの旅行保険は何をカバーしていますか？

--- 使用されたコンテキスト ---
取得されたコンテキスト:
Document: Contosoの旅行保険は、医療緊急事態、旅行キャンセル、手荷物紛失をカバーしています。
Metadata: {'source': 'training', 'type': 'explanation'}

Document: 当社のプレミアム旅行サービスには、個別の旅程計画と24時間365日のコンシェルジュサポートが含まれています。
Metadata: {'source': 'training', 'type': 'explanation'}


------------------------

応答: Contosoの旅行保険は、医療緊急事態、旅行キャンセル、手荷物紛失をカバーしています。



クエリ: シドニーの天気はどうですか？
場所: sydney

--- 使用されたコンテキスト ---
取得されたコンテキスト:
Document: Contoso Travelは世界中のエキゾチックな目的地への高級バケーションパッケージを提供しています。
Metadata: {'source': 'training', 'type': 'explanation'}

Document: 当社のプレミアム旅行サービスには、個別の旅程計画と24時間365日のコンシェルジュサポートが含まれています。
Metadata: {'type': 'explanation', 'source': 'training'}


sydneyの天気情報:
Sydneyの天気:
気温: 27°C
天候: 快晴
湿度: 45%
風速: 時速19km
------------------------

応答: シドニーの天気は快晴で、気温は27°C、湿度は45%、風速は時速19kmです。



クエリ: Contosoが提供する高級な目的地は何ですか？パリの天気も教えてください。
場所: paris

--- 使用されたコンテキスト ---
取得されたコンテキスト:
Document: Contoso Travelは世界中のエキゾチックな目的地への高級バケーションパッケージを提供しています。
Metad

## 🎓 学習完了！次のステップ

### 📖 次に学習が推奨されるノートブック

このRAGシステムの基礎を習得したら、以下の高度なトピックに進むことをお勧めします：

#### 🔗 関連学習リンク

1. **信頼できるエージェントの構築**
   - [06-building-trustworthy-agents](../../06-building-trustworthy-agents/README.md)
   - RAGシステムの信頼性向上、エラー処理、セキュリティ対策

2. **計画・設計パターン**
   - [07-planning-design](../../07-planning-design/README.md)
   - より複雑なエージェントシステムの設計手法

3. **マルチエージェントシステム**
   - [08-multi-agent](../../08-multi-agent/README.md)
   - 複数のエージェントが協調して問題解決を行うシステム

4. **本番環境でのAIエージェント**
   - [10-ai-agents-production](../../10-ai-agents-production/README.md)
   - スケーラブルで堅牢なAIエージェントシステムの構築

### 🚀 発展的な学習

RAGシステムをさらに発展させたい場合は：
- **カスタムツール開発**: [04-tool-use](../../04-tool-use/README.md) でツール活用手法を学習
- **メタ認知**: [09-metacognition](../../09-metacognition/README.md) でエージェントの自己認識能力を向上

---
**Happy Learning! 🎉** 次のレベルのAIエージェント開発にチャレンジしてください！